# M2I EZ-ZAKKAR MOHAMMED

# Script pour la reconnaisance faciale :

<p>Le script suivant permet de reconnaitre le visage de n'import quelle personne à partir de s'image.<br>
Dans les parties suivantes on va expliquer en détails le fonctionment de ce script.</p><br>
<ul>
<li>Étape 1 : Les recommandations des bibliothèques</li>
<li>Étape 2 : Entrainement</li>
<li>Étape 3 : Test</li>
</ul>

## Étape 1 : Les recommandations des bibliothèques

In [1]:
import cv2
import face_recognition
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this


# Initialize some variables
known_face_encodings = []
known_face_names = []

## Étape 2 : Entrainement (Training) 

In [2]:
# Training function
def Training(image):
    
    small_frame = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # If a match was found in known_face_encodings, just use the first one.
        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        face_names.append(name)

        
    nbr_face = 0
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        nbr_face += 1
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        img = image[top:bottom, left:right]

        img = Image.fromarray(np.uint8(img)).convert('RGB')
        
        #img = img.resize((200,200))

        root = tk.Tk()
        # set the title
        nameDialog = "Image : "+str(nbr_face)
        root.title(nameDialog)
        # set the image
        img = ImageTk.PhotoImage(img)
        
        imageL = tk.Label(root, image=img)
        imageL.pack()
        
        label = tk.Label(root, text="Name")
        label.pack(side=tk.TOP, padx=5, pady=5)
        # set the Entry
        entry = tk.Entry(root)
        entry.pack(side=tk.TOP, padx=5, pady=5)
        entry.insert(0,face_names[nbr_face-1])
        
        buttonCancel = tk.Button(root, text='Cancel',
           command=root.quit)
        buttonCancel.pack(side=tk.LEFT, padx=5, pady=5)
        buttonAdd = tk.Button(root, text='  Add  ',
           command=(lambda r=root : addFace(r, entry.get(), face_encodings[nbr_face-1])))
        buttonAdd.pack(side=tk.LEFT, padx=5, pady=5)
        
        # display GUI
        root.mainloop()
        try :
            root.destroy()
        except:
            # nothing
            root.quit()
    
    # return nbr of the faces in the image
    return nbr_face

# add encoding face and name
def addFace(root, name, face_enc):
    if name == "":
        name = "Unknown"
    known_face_encodings.append(face_enc)
    known_face_names.append(name)
    root.quit()
    return 0 

Entrainer le script à partir des images locales.

In [3]:
def trainingWithImage():
    root = tk.Tk()
    root.withdraw()
    # Load picture
    file_path = filedialog.askopenfilename()
    root.quit    # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
    if file_path:
        
        image = face_recognition.load_image_file(file_path)
        
        # call Training function
        nbr = Training(image)
        print(nbr, "faces found in the selected image")
    

In [4]:
# Call trainingWithImage function
trainingWithImage()

Entrainer le script à partir des photos prises avec la webcam.

In [4]:
def trainingWithWebcam():
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(0)
    try :
        while True:

            ret, frame = video_capture.read()

            # Display the resulting image
            cv2.imshow('Video', frame)
            # Hit 'q' on the keyboard to quit! Or Hit 'p' to take photo
            key = cv2.waitKey(1)
            if key == ord('q'): # Hit 'q' on the keyboard to quit!
                cv2.destroyAllWindows()
                #return
            if key == ord('p'): # Hit 'p' on the keyboard to take photo!
                cv2.destroyAllWindows()
                # call Training function
                nbr = Training(frame)
                print(nbr, "faces found in the selected image")
                break
    except:
        video_capture.release()
                   
    # Release handle to the webcam
    video_capture.release()


In [6]:
# Call trainingWithWebca function
trainingWithWebcam()

1 faces found in the selected image


## Étape 3 : Test

Dans cette partie nous essayons de tester le script en utilisant la webcam.

In [7]:
# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.


# Define Test function
def Test():
    
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(0)
    
    process_this_frame = True
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # If a match was found in known_face_encodings, just use the first one.
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            
            if name == "Unknown":
                color = (0, 0, 255)
            else : 
                color = (0, 255, 0)
                
            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [8]:
# Call Test function
Test()